In [44]:
#codage source UTF-8


def text_to_binary(text):
    return ' '.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary):
    return ''.join(chr(int(b, 2)) for b in binary.split())

# Exemple d'utilisation
message = "Hello!"
binary_message = text_to_binary(message)
decoded_message = binary_to_text(binary_message)

print("Texte original :", message)
print("Texte en binaire :", binary_message)
print("Texte décodé :", decoded_message)


Texte original : Hello!
Texte en binaire : 01001000 01100101 01101100 01101100 01101111 00100001
Texte décodé : Hello!


In [33]:
import numpy as np
import pyldpc
import tensorflow as tf
import nltk
import random
import string

# Télécharger le dictionnaire de mots français
nltk.download('words')
from nltk.corpus import words

# Sélection des mots français
mots_francais = [mot.lower() for mot in words.words() if mot.isalpha()]

# Paramètres du codage LDPC
n = 64  # Longueur du bloc LDPC codé
k = 32  # Longueur du message original avant codage
# Création de la matrice de contrôle de parité H
d_v = 2  # Degré des colonnes
d_c = 4  # Degré des lignes
H = pyldpc.create_parity_check_matrix(n, d_v, d_c, systematic=True)
# Calcul de la matrice génératrice G
G = pyldpc.compute_generator_matrix(H)

# Plage de SNR et nombre d'échantillons
snr_db_range = np.arange(0, 11, 2)
nombre_echantillons_par_snr = 1000

# Base de données
train_data_bruitee = []
train_data_originale = []

# Fonction pour convertir un mot en binaire (ASCII)
def mot_en_binaire(mot):
    binaire = ''.join(format(ord(c), '08b') for c in mot)  # ASCII en binaire
    return np.array([int(b) for b in binaire])

# Génération de la base de données
for snr_db in snr_db_range:
    print(f"📡 Génération des données pour SNR = {snr_db} dB...")
    
    for _ in range(nombre_echantillons_par_snr):
        # 1. Choisir un mot aléatoire en français
        mot_original = random.choice(mots_francais)
        
        # 2. Convertir en binaire
        mot_original_binaire = mot_en_binaire(mot_original)

        # 3. Adapter à la taille k
        if len(mot_original_binaire) < k:
            mot_original_binaire = np.pad(mot_original_binaire, (0, k - len(mot_original_binaire)), 'constant')
        elif len(mot_original_binaire) > k:
            mot_original_binaire = mot_original_binaire[:k]

        mot_original_binaire_float = mot_original_binaire.astype(np.float64)

        # 4. Encodage LDPC
        mot_code_ldpc = pyldpc.encode(G, mot_original_binaire_float)

        # 5. Ajout du bruit (AWGN)
        signal_puissance = np.mean(mot_code_ldpc ** 2)
        snr_lin = 10 ** (snr_db / 10)
        bruit_puissance = signal_puissance / snr_lin
        bruit_std = np.sqrt(bruit_puissance)
        bruit_gaussien = np.random.normal(0, bruit_std, size=len(mot_code_ldpc))
        
        mot_code_bruite = mot_code_ldpc + bruit_gaussien

        # 6. Stockage
        train_data_bruitee.append(mot_code_bruite)
        train_data_originale.append(mot_original_binaire_float)

# Conversion en NumPy
train_data_bruitee = np.array(train_data_bruitee)
train_data_originale = np.array(train_data_originale)

# Sauvegarde en .npy
np.save("ldpc_data_bruitee.npy", train_data_bruitee)
np.save("ldpc_data_originale.npy", train_data_originale)

# Vérification
print("\n✅ Base de données générée et sauvegardée avec succès !")
print(f"Nombre total d'échantillons : {len(train_data_bruitee)}")
print(f"Forme des données bruitées : {train_data_bruitee.shape}")
print(f"Forme des données originales : {train_data_originale.shape}")

# Exemple d'affichage
print("\n🔍 Exemples de données :")
for i in range(2):  # Afficher 2 échantillons
    print(f"\nTrame bruitée [{i}]: {train_data_bruitee[i][:10]} ...")  # Affiche les 10 premiers éléments
    print(f"Trame originale [{i}]: {train_data_originale[i]}")  


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\CyberVortex\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


AttributeError: module 'pyldpc' has no attribute 'create_parity_check_matrix'

In [29]:
import numpy as np

# Chargement des données
X = np.load("ldpc_data_bruitee.npy")
Y = np.load("ldpc_data_originale.npy")

print("Exemple de trame bruitée :", X[0])
print("Exemple de trame originale :", Y[0])


FileNotFoundError: [Errno 2] No such file or directory: 'ldpc_data_bruitee.npy'

In [34]:
import numpy as np
import ldpc.codes
from ldpc.bplsd_decoder import BpLsdDecoder
import pickle

# 🔹 1. Génération de la matrice de contrôle de parité H
H = ldpc.codes.hamming_code(5)  # Code de Hamming (31,26) => n=31, k=26
n = H.shape[1]  # Longueur du mot de code
k = n - H.shape[0]  # Longueur du message original

# 🔹 2. Initialisation du décodeur BP-LSD
decoder = BpLsdDecoder(
    H,
    error_rate=0.1,
    bp_method='product_sum',
    max_iter=10,
    schedule='serial',
    lsd_method='lsd_cs',
    lsd_order=0
)

# 🔹 3. Paramètres du bruit et de la base de données
snr_db_range = np.arange(0, 11, 2)  # SNR de 0dB à 10dB
nb_samples = 1000  # Nombre d'échantillons

# 🔹 4. Listes pour stocker les données
dataset = []

# 🔹 5. Génération des échantillons
for snr_db in snr_db_range:
    print(f"Génération des données pour SNR = {snr_db} dB...")
    
    for _ in range(nb_samples):
        # 🟢 1. Générer un message binaire aléatoire
        msg = np.random.randint(0, 2, k, dtype=np.uint8)
        
        # 🟢 2. Encodage LDPC
        codeword = ldpc.codes.encode(H, msg)
        
        # 🟢 3. Ajout du bruit AWGN
        snr_lin = 10**(snr_db / 10)  # Conversion dB -> linéaire
        noise_std = np.sqrt(1 / (2 * snr_lin))  # Écart-type du bruit
        noise = np.random.normal(0, noise_std, codeword.shape)
        noisy_codeword = codeword + noise
        
        # 🟢 4. Stockage des données
        dataset.append((noisy_codeword, msg))

# 🔹 6. Exportation de la base de données
with open("ldpc_dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)

print("\n✅ Base de données générée et sauvegardée sous 'ldpc_dataset.pkl' !")


Génération des données pour SNR = 0 dB...


AttributeError: module 'ldpc.codes' has no attribute 'encode'

In [36]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message
import pickle

# 🔹 1. Définition des paramètres LDPC
n = 15  # Longueur du mot de code
d_v = 4  # Degré des nœuds de variable
d_c = 5  # Degré des nœuds de contrôle
snr_db_range = np.arange(0, 11, 2)  # SNR de 0dB à 10dB
nb_samples = 1000  # Nombre d'échantillons

# 🔹 2. Génération des matrices LDPC
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]  # Taille du message original

# 🔹 3. Base de données
dataset = []

# 🔹 4. Génération des échantillons
for snr_db in snr_db_range:
    print(f"Génération des données pour SNR = {snr_db} dB...")
    
    for _ in range(nb_samples):
        # 🟢 1. Générer un message binaire aléatoire
        v = np.random.randint(2, size=k)

        # 🟢 2. Encodage LDPC
        y = encode(G, v, snr_db)

        # 🟢 3. Décodage
        d = decode(H, y, snr_db)

        # 🟢 4. Extraction du message original
        x = get_message(G, d)

        # 🟢 5. Stockage
        dataset.append((y, v))

# 🔹 5. Sauvegarde de la base de données
with open("ldpc_dataset.csv", "wb") as f:
    pickle.dump(dataset, f)

print("\n✅ Base de données sauvegardée sous 'ldpc_dataset.pkl' !")


Génération des données pour SNR = 0 dB...
Génération des données pour SNR = 2 dB...
Génération des données pour SNR = 4 dB...
Génération des données pour SNR = 6 dB...
Génération des données pour SNR = 8 dB...
Génération des données pour SNR = 10 dB...

✅ Base de données sauvegardée sous 'ldpc_dataset.pkl' !


In [39]:
import numpy as np
import pandas as pd
from pyldpc import make_ldpc, encode, decode, get_message

# Paramètres du codage LDPC
n = 15  # Longueur du bloc codé
d_v = 4  # Degré des variables
d_c = 5  # Degré des contrôles
snr = 20  # SNR en dB
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)

k = G.shape[1]  # Longueur du message original
longueur_min_mot = 10
longueur_max_mot = 50

# Nombre d'échantillons
nombre_echantillons = 1000

# Listes pour stocker les données
train_data_bruitee = []
train_data_originale = []

# Génération des échantillons
for _ in range(nombre_echantillons):
    # 1. Générer un mot binaire original aléatoire de longueur variable
    longueur_mot_original = np.random.randint(longueur_min_mot, longueur_max_mot + 1)
    mot_original_binaire = np.random.randint(0, 2, longueur_mot_original)
    
    # 2. Padding du mot original pour correspondre à la longueur k
    if len(mot_original_binaire) < k:
        mot_original_binaire = np.pad(mot_original_binaire, (0, k - len(mot_original_binaire)), 'constant')
    elif len(mot_original_binaire) > k:
        mot_original_binaire = mot_original_binaire[:k]  # Tronquer si plus long que k

    mot_original_binaire_float = mot_original_binaire.astype(np.float64)  # Conversion en float

    # 3. Encodage LDPC
    mot_code_ldpc = encode(G, mot_original_binaire)

    # 4. Ajouter du bruit gaussien
    signal_puissance = np.mean(mot_code_ldpc**2)
    snr_lin = 10**(snr/10)
    bruit_puissance = signal_puissance / snr_lin
    bruit_std = np.sqrt(bruit_puissance)
    bruit_gaussien = np.random.normal(0, bruit_std, size=len(mot_code_ldpc))

    # 5. Ajouter le bruit au mot codé
    mot_code_bruite = mot_code_ldpc + bruit_gaussien

    # 6. Stocker les données dans les listes
    train_data_bruitee.append(mot_code_bruite)
    train_data_originale.append(mot_original_binaire_float)

# Conversion des listes en tableaux NumPy pour pouvoir les manipuler facilement
train_data_bruitee = np.array(train_data_bruitee)
train_data_originale = np.array(train_data_originale)

# Conversion en DataFrame pour faciliter l'exportation en CSV
data_dict = {
    "Trame codée bruitée": [', '.join(map(str, frame)) for frame in train_data_bruitee],
    "Trame binaire originale": [', '.join(map(str, frame)) for frame in train_data_originale]
}

df = pd.DataFrame(data_dict)

# Exporter le DataFrame au format CSV
df.to_csv("base_de_donnees_ldpc.csv", index=False)

print("\nBase de données générée et exportée en CSV avec succès!")


TypeError: encode() missing 1 required positional argument: 'snr'

In [41]:
import numpy as np
import pandas as pd
from pyldpc import make_ldpc, encode
import random
import string
from sklearn.model_selection import train_test_split

# Définir les paramètres pour la génération de la base de données
n = 15   # Longueur du bloc codé LDPC
dv = 4   # Degré des nœuds variables pour LDPC
dc = 5   # Degré des nœuds de contrôle pour LDPC
snr_db = 5  # SNR en dB pour le bruit gaussien (vous pourrez varier cela)
nombre_echantillons = 2000  # Nombre total d'échantillons à générer
test_size = 0.2  # Proportion des données pour l'ensemble de test
validation_size = 0.2 # Proportion des données pour l'ensemble de validation (après avoir séparé le test)


# Générer les matrices LDPC une seule fois (en dehors de la boucle)
H, G = make_ldpc(n, dv, dc, systematic=True, sparse=True)
k = G.shape[1]  # Longueur du message original (bits d'information)

# Fonction pour générer une trame binaire aléatoire de longueur k (pour l'entraînement DL, on utilise des trames binaires fixes)
def generate_binary_frame(length):
    return np.random.randint(0, 2, size=length, dtype=np.uint8) # directement en uint8 pour efficacité

# Fonction pour ajouter du bruit gaussien
def add_gaussian_noise(signal, snr_db):
    signal_power = np.mean(signal**2)
    snr_linear = 10**(snr_db/10)
    noise_power = signal_power / snr_linear
    noise_std = np.sqrt(noise_power)
    noise = np.random.normal(0, noise_std, size=len(signal))
    return signal + noise

# Listes pour stocker les données
input_data_noisy_frames = [] # Pour les trames codées bruitées (entrée du décodeur DL)
output_data_original_frames = [] # Pour les trames binaires originales (sortie cible du décodeur DL)
labels = [] # Pour les mots originaux (si vous souhaitez garder les mots comme labels)

## Génération des échantillons
for i in range(nombre_echantillons):
    # 1. Générer une trame binaire aléatoire de longueur k (bits d'information)
    message_original_binaire = generate_binary_frame(k)

    # 2. Encodage LDPC
    mot_code_ldpc = encode(G, message_original_binaire) # Argument snr supprimé ici

    # 3. Ajouter du bruit gaussien
    mot_code_bruite = add_gaussian_noise(mot_code_ldpc, snr_db)

    # 4. Stocker les données (utiliser des types NumPy directement pour efficacité)
    input_data_noisy_frames.append(mot_code_bruite)
    output_data_original_frames.append(message_original_binaire)
    labels.append(f"sample_{i+1}") # Etiquette simple pour chaque échantillon
    
# Conversion des listes en tableaux NumPy (déjà fait en append mais redondant pour s'assurer)
input_data_noisy_frames = np.array(input_data_noisy_frames)
output_data_original_frames = np.array(output_data_original_frames)

# Séparation des données en ensembles d'entraînement, validation et test
# 1. Séparer d'abord l'ensemble de test
train_val_input, test_input, train_val_output, test_output = train_test_split(
    input_data_noisy_frames, output_data_original_frames, test_size=test_size, random_state=42
)

# 2. Séparer l'ensemble d'entraînement et de validation à partir du reste (train_val)
train_input, val_input, train_output, val_output = train_test_split(
    train_val_input, train_val_output, test_size=validation_size / (1 - test_size), random_state=42 # Ajuster la taille de validation
)

# Créer des DataFrames pandas pour chaque ensemble pour faciliter l'exportation en CSV (optionnel, peut être sauvegardé directement en .npy)
def create_dataframe(input_frames, output_frames, set_name):
    data_dict = {
        "Trame codée bruitée": [', '.join(map(str, frame)) for frame in input_frames],
        "Trame binaire originale": [', '.join(map(str, frame)) for frame in output_frames]
    }
    df = pd.DataFrame(data_dict)
    return df

train_df = create_dataframe(train_input, train_output, "train")
val_df = create_dataframe(val_input, val_output, "validation")
test_df = create_dataframe(test_input, test_output, "test")


# Exporter les DataFrames au format CSV dans des dossiers séparés
import os
output_dir = "base_de_donnees_ldpc_binaire"
os.makedirs(output_dir, exist_ok=True) # Créer le dossier principal si inexistant

train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "validation")
test_dir = os.path.join(output_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


train_df.to_csv(os.path.join(train_dir, "train_data_ldpc_binaire.csv"), index=False)
val_df.to_csv(os.path.join(val_dir, "validation_data_ldpc_binaire.csv"), index=False)
test_df.to_csv(os.path.join(test_dir, "test_data_ldpc_binaire.csv"), index=False)


# (Option alternative pour sauvegarder en .npy directement - plus efficace pour les données numériques pour DL)
# np.save(os.path.join(train_dir, "train_input_noisy.npy"), train_input)
# np.save(os.path.join(train_dir, "train_output_original.npy"), train_output)
# np.save(os.path.join(val_dir, "val_input_noisy.npy"), val_input)
# np.save(os.path.join(val_dir, "val_output_original.npy"), val_output)
# np.save(os.path.join(test_dir, "test_input_noisy.npy"), test_input)
# np.save(os.path.join(test_dir, "test_output_original.npy"), test_output)


print(f"\nBase de données binaire LDPC générée avec {nombre_echantillons} échantillons.")
print(f"Ensembles de données CSV exportés dans les dossiers: '{output_dir}/train', '{output_dir}/validation', '{output_dir}/test'")
print(f"Paramètres utilisés: Longueur bloc codé n={n}, Longueur message k={k}, SNR={snr_db}dB")
print(f"Répartition: Train={len(train_input)}, Validation={len(val_input)}, Test={len(test_input)}")

TypeError: encode() missing 1 required positional argument: 'snr'

In [42]:
import numpy as np
import pandas as pd
from pyldpc import make_ldpc, encode
import random
import string

# Paramètres du codage LDPC
n = 15  # Longueur du bloc codé
d_v = 4  # Degré des variables
d_c = 5  # Degré des contrôles
snr = 20  # SNR en dB
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)

k = G.shape[1]  # Longueur du message original

# Fonction pour générer un mot aléatoire de longueur variable
def random_word(length):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

# Fonction pour convertir un mot en binaire (UTF-8 ou ASCII)
def word_to_binary(word, encoding="utf-8"):
    return np.array([int(bit) for bit in ''.join(format(ord(char), '08b') for char in word)], dtype=np.uint8)

# Longueur des mots aléatoires (entre 0 et 10)
longueur_min_mot = 0
longueur_max_mot = 10

# Nombre d'échantillons
nombre_echantillons = 1000

# Listes pour stocker les données
train_data_bruitee = []
train_data_originale = []
labels = []

# Génération des échantillons
for _ in range(nombre_echantillons):
    # 1. Générer un mot aléatoire de longueur entre 0 et 10
    longueur_mot = random.randint(longueur_min_mot, longueur_max_mot)
    mot_original = random_word(longueur_mot)
    
    # 2. Convertir le mot en binaire
    mot_original_binaire = word_to_binary(mot_original)
    
    # 3. Padding du mot original pour correspondre à la longueur k
    if len(mot_original_binaire) < k:
        mot_original_binaire = np.pad(mot_original_binaire, (0, k - len(mot_original_binaire)), 'constant')
    elif len(mot_original_binaire) > k:
        mot_original_binaire = mot_original_binaire[:k]  # Tronquer si plus long que k
    
    mot_original_binaire_float = mot_original_binaire.astype(np.float64)  # Conversion en float

    # 4. Encodage LDPC
    mot_code_ldpc = encode(G, mot_original_binaire, snr)

    # 5. Ajouter du bruit gaussien
    signal_puissance = np.mean(mot_code_ldpc**2)
    snr_lin = 10**(snr/10)
    bruit_puissance = signal_puissance / snr_lin
    bruit_std = np.sqrt(bruit_puissance)
    bruit_gaussien = np.random.normal(0, bruit_std, size=len(mot_code_ldpc))

    # 6. Ajouter le bruit au mot codé
    mot_code_bruite = mot_code_ldpc + bruit_gaussien

    # 7. Stocker les données dans les listes
    train_data_bruitee.append(mot_code_bruite)
    train_data_originale.append(mot_original_binaire_float)
    labels.append(mot_original)  # Ajouter l'étiquette

# Conversion des listes en tableaux NumPy pour pouvoir les manipuler facilement
train_data_bruitee = np.array(train_data_bruitee)
train_data_originale = np.array(train_data_originale)

# Conversion en DataFrame pour faciliter l'exportation en CSV
data_dict = {
    "Mot original": labels,  # Mot original comme étiquette
    "Trame codée bruitée": [', '.join(map(str, frame)) for frame in train_data_bruitee],
    "Trame binaire originale": [', '.join(map(str, frame)) for frame in train_data_originale]
}

df = pd.DataFrame(data_dict)

# Exporter le DataFrame au format CSV
df.to_csv("base_de_donnees_ldpc_mots_aleatoires.csv", index=False)

print("\nBase de données générée et exportée en CSV avec succès!")



Base de données générée et exportée en CSV avec succès!
